The create() function takes at least two arguments, a name for the newly created class and a base class. Any subsequent argument becomes an attribute of the class. As specified in the Fitness documentation, the weights attribute must be a tuple so that multi-objective and single objective fitnesses can be treated the same way. A 

In [1]:
#importação da biblioteca DEAP dos recursos necessários
from deap import base, creator

#para um problema de minimização
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
#deriva de uma lista de fitness do recurso acima
creator.create("Individual", list, fitness=creator.FitnessMin)

In [2]:
#FitnessMulti would be created the same way but using:
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, 1.0))
#This code produces a fitness that minimizes the first objective and maximize the second one.

In [3]:
import random

from deap import base, creator, tools
import numpy as numpy
import array as array


creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual1", list, fitness=creator.FitnessMax)
creator.create("Individual2", array.array, typecode="d", fitness=creator.FitnessMax)
creator.create("Individual3", numpy.ndarray, fitness=creator.FitnessMax)

IND_SIZE=10

toolbox = base.Toolbox()
toolbox.register("attr_float", random.random)
toolbox.register("individual1", tools.initRepeat, creator.Individual1,
                 toolbox.attr_float, n=IND_SIZE)
toolbox.register("individual2", tools.initRepeat, creator.Individual2,
                 toolbox.attr_float, n=IND_SIZE)
toolbox.register("individual3", tools.initRepeat, creator.Individual3,
                 toolbox.attr_float, n=IND_SIZE)

##### List of Floats

The first individual created will be a simple list containing floats.

In [4]:
toolbox.individual1()

[0.24643857765123323,
 0.9516494784766419,
 0.7989717651919789,
 0.25925269588032307,
 0.9919646121954666,
 0.06559946046175003,
 0.32037055112775104,
 0.13328421628239095,
 0.42393926585262076,
 0.2749639983686878]

In [5]:
toolbox.individual2()

array('d', [0.06602803258586154, 0.5627691778866947, 0.9671355367687625, 0.668847458248458, 0.08646868722319345, 0.1621970396340453, 0.8170265948724235, 0.5653176193262146, 0.613396932911355, 0.7594473829559393])

In [6]:
toolbox.individual3()

Individual3([ 0.10986756,  0.70350512,  0.58799458,  0.11305006,  0.58438031,
        0.9861501 ,  0.62776642,  0.18412598,  0.69716903,  0.4219705 ])

##### Permutation

An individual for the permutation representation is almost similar to the general list individual. In fact they both inherit from the basic list type. The only difference is that instead of filling the list with a series of floats, we need to generate a random permutation and provide that permutation to the individual.

In [7]:
import random

from deap import base
from deap import creator
from deap import tools

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

IND_SIZE=10

toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(IND_SIZE), IND_SIZE)
toolbox.register("individual", tools.initIterate, creator.Individual,
                 toolbox.indices)

In [8]:
toolbox.individual()

[1, 6, 4, 5, 2, 7, 3, 0, 8, 9]

##### Arithmetic Expression

The next individual that is commonly used is a prefix tree of mathematical expressions.

In [9]:
import operator

from deap import base
from deap import creator
from deap import gp
from deap import tools

pset = gp.PrimitiveSet("MAIN", arity=1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin,
               pset=pset)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual,
                 toolbox.expr)

In [10]:
toolbox.individual()

#### Evolution Strategy

Evolution strategies individuals are slightly different as they contain generally two lists, one for the actual individual and one for its mutation parameters.

In [11]:
import array
import random

from deap import base
from deap import creator
from deap import tools

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", array.array, typecode="d",
               fitness=creator.FitnessMin, strategy=None)
creator.create("Strategy", array.array, typecode="d")

def initES(icls, scls, size, imin, imax, smin, smax):
    ind = icls(random.uniform(imin, imax) for _ in range(size))
    ind.strategy = scls(random.uniform(smin, smax) for _ in range(size))
    return ind

IND_SIZE = 10
MIN_VALUE, MAX_VALUE = -5., 5.
MIN_STRAT, MAX_STRAT = -1., 1. 

toolbox = base.Toolbox()
toolbox.register("individual", initES, creator.Individual,
                 creator.Strategy, IND_SIZE, MIN_VALUE, MAX_VALUE, MIN_STRAT, 
                 MAX_STRAT)

In [12]:
toolbox.individual()

array('d', [3.164859469264206, -1.0032138039766414, 0.8460881895156733, -4.581236028341523, -2.0770072018691965, -1.4317873323199293, -4.471546962437544, -0.9285804111393334, 2.9024705645121305, 0.07074657292366204])

#### Particle

A particle is another special type of individual as it usually has a speed and generally remember its best position.

In [13]:
import random

from deap import base
from deap import creator
from deap import tools

creator.create("FitnessMax", base.Fitness, weights=(1.0, 1.0))
creator.create("Particle", list, fitness=creator.FitnessMax, speed=None,
               smin=None, smax=None, best=None)

def initParticle(pcls, size, pmin, pmax, smin, smax):
    part = pcls(random.uniform(pmin, pmax) for _ in range(size))
    part.speed = [random.uniform(smin, smax) for _ in range(size)]
    part.smin = smin
    part.smax = smax
    return part

toolbox = base.Toolbox()
toolbox.register("particle", initParticle, creator.Particle, size=2,
                 pmin=-6, pmax=6, smin=-3, smax=3)

In [14]:
toolbox.particle()

[-3.7874751809877925, -3.226685717214785]

In [15]:
#A Funky One

import random

from deap import base
from deap import creator
from deap import tools

creator.create("FitnessMax", base.Fitness, weights=(1.0, 1.0))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

INT_MIN, INT_MAX = 5, 10
FLT_MIN, FLT_MAX = -0.2, 0.8
N_CYCLES = 4

toolbox.register("attr_int", random.randint, INT_MIN, INT_MAX)
toolbox.register("attr_flt", random.uniform, FLT_MIN, FLT_MAX)
toolbox.register("individual", tools.initCycle, creator.Individual,
                 (toolbox.attr_int, toolbox.attr_flt), n=N_CYCLES)

In [16]:
toolbox.individual()

[8,
 0.41975149506497816,
 10,
 0.15853458732325515,
 9,
 0.6993836289899276,
 6,
 0.4922740442185129]

### Population

Populations are much like individuals. Instead of being initialized with attributes, they are filled with individuals, strategies or particles.

In [17]:
#Bag
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.population(n=3)

[[6,
  0.25281096771250416,
  8,
  0.05803133195448856,
  6,
  0.2482148733365625,
  10,
  0.7124757756904154],
 [7,
  0.15481569755092833,
  5,
  0.6207432946708353,
  8,
  0.7515881651546796,
  6,
  0.7629323525347351],
 [7,
  0.7841224499374515,
  10,
  0.0767901394814699,
  7,
  0.7174858402569939,
  5,
  0.29488235792637446]]

In [18]:
#Grid
N_COL, N_ROW = 2, 2

toolbox.register("row", tools.initRepeat, list, toolbox.individual, n=N_COL)
toolbox.register("population", tools.initRepeat, list, toolbox.row, n=N_ROW)

toolbox.population()

[[[6,
   0.13788129927446185,
   8,
   0.5013172131930987,
   5,
   0.5614584384924746,
   6,
   0.3777796046885797],
  [9,
   -0.1284952842520402,
   8,
   0.46996553228634347,
   8,
   0.16438948589283703,
   8,
   0.524916527688815]],
 [[10,
   -0.0836855038568966,
   8,
   0.545858794296407,
   5,
   0.2284120558965262,
   9,
   0.3398004167176258],
  [7,
   0.44705910315984826,
   6,
   0.25216994052172,
   5,
   0.5451282565952886,
   7,
   0.02806134237004415]]]

In [19]:
#Swarm
creator.create("Swarm", list, gbest=None, gbestfit=creator.FitnessMax)
toolbox.register("swarm", tools.initRepeat, creator.Swarm, toolbox.particle)

toolbox.swarm()

AttributeError: 'Toolbox' object has no attribute 'particle'

In [ ]:
toolbox.register("deme", tools.initRepeat, list, toolbox.individual)

DEME_SIZES = 10, 50, 100
population = [toolbox.deme(5) for i in DEME_SIZES]

toolbox.deme()

#### Seeding a Population

Sometimes, a first guess population can be used to initialize an evolutionary algorithm. The key idea to initialize a population with not random individuals is to have an individual initializer that takes a content as argument.

In [ ]:
import json

from deap import base
from deap import creator

creator.create("FitnessMax", base.Fitness, weights=(1.0, 1.0))
creator.create("Individual", list, fitness=creator.FitnessMax)

def initIndividual(icls, content):
    return icls(content)

def initPopulation(pcls, ind_init, filename):
    with open(filename, "r") as pop_file:
        contents = json.load(pop_file)
    return pcls(ind_init(c) for c in contents)

toolbox = base.Toolbox()

toolbox.register("individual_guess", initIndividual, creator.Individual)
#toolbox.register("population_guess", initPopulation, list, toolbox.individual_guess, "my_guess.json")
toolbox.register("population_guess", initPopulation, list, creator.Individual, "my_guess.json")

population = toolbox.population_guess()